In [0]:
%%bash
[[ -z "$COLAB_GPU" ]] && exit 0
apt install subversion >> log
pip install pyarts typhon >> log
svn co -q https://arts.mi.uni-hamburg.de/svn/rt/arts-xml-data/trunk/spectroscopy/Perrin/ >> log

In [0]:
"""Calculate and plot absorption cross sections."""
import glob
from os.path import basename

from ipywidgets import interactive
import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import numpy as np
from pyarts.workspace import Workspace
from typhon.plots import styles
from typhon.utils import Timer

plt.style.use(styles("typhon"))

class ArtsCalc:
    _nfrequencies = -1
    _species = ""
    _ws = None
    _ymax = None

    def plot_absorption(self, temperature=300, pressure=800, species="H2O", nfrequencies=5000):
        # Define parameters
        verbosity = 0
        pressure *= 100
    
        # Call ARTS to calculate absorption cross sections
        if nfrequencies != self._nfrequencies or species != self._species:
            self._species = species
            self._nfrequencies = nfrequencies
            self._ws = setup_absxsec(species, fnum=nfrequencies, verbosity=verbosity)
            self._ymax = None

        freq, abs_xsec = calculate_absxsec(self._ws, pressure, temperature)
        if self._ymax is None:
          self._ymax = np.max(abs_xsec)
    
        # Plot the results.
    
        fig, ax = plt.subplots()
        plt.subplots_adjust(bottom=0.25)
    
        ax.plot(freq / 1e9, abs_xsec, rasterized=True)
    
        ax.set_xlim(freq.min() / 1e9, freq.max() / 1e9)
        ax.set_ylim(0, self._ymax)
        ax.set_xlabel("Frequency [GHz]")
        ax.set_ylabel(r"Abs. cross section [$\sf m^2$]")
        ax.set_title(f"{species} p:{pressure/100} hPa T:{temperature:0.0f} K")
    
        plt.show()  # Open an interactive figure


def setup_absxsec(
    species="N2O",
    fmin=10e9,
    fmax=2000e9,
    fnum=10_000,
    lineshape="LP",
    normalization="RQ",
    verbosity=0,
):
    """Setup absorption cross sections.

    Parameters:
        species (str): Absorption species name.
        fmin (float): Minimum frequency [Hz].
        fmax (float): Maximum frequency [Hz].
        fnum (int): Number of frequency grid points.
        lineshape (str): Line shape model.
        normalization (str): Line shape normalization factor.
        verbosity (int): Set ARTS verbosity (``0`` prevents all output).

    Returns:
        Workspace: ARTS Workspace.
    """
    # Create ARTS workspace and load default settings
    ws = Workspace(verbosity=verbosity)
    ws.execute_controlfile("general/general.arts")
    ws.execute_controlfile("general/continua.arts")
    ws.execute_controlfile("general/agendas.arts")
    ws.verbositySetScreen(ws.verbosity, verbosity)

    # We do not want to calculate the Jacobian Matrix
    ws.jacobianOff()

    # Agenda for scalar gas absorption calculation
    ws.Copy(ws.abs_xsec_agenda, ws.abs_xsec_agenda__noCIA)

    # Define absorption species
    ws.abs_speciesSet(species=[species])
    ws.ArrayOfIndexSet(ws.abs_species_active, [0])

    # Line catalogue: Perrin or HITRAN
    ws.ReadSplitARTSCAT(
        basename="./Perrin/",
        fmin=0.9 * fmin,
        fmax=1.1 * fmax,
        globalquantumnumbers="",
        localquantumnumbers="",
        ignore_missing=0,
    )
    ws.abs_lines_per_speciesCreateFromLines()

    # Set the lineshape function for all calculated tags
    ws.abs_linesSetLineShapeType(ws.abs_lines, lineshape)
    ws.abs_linesSetCutoff(ws.abs_lines, "None", 0.0)
    ws.abs_linesSetNormalization(ws.abs_lines, normalization)

    # isotop
    ws.isotopologue_ratiosInitFromBuiltin()

    # Atmospheric settings
    ws.AtmosphereSet1D()

    # Create a frequency grid
    ws.VectorNLinSpace(ws.f_grid, fnum, fmin, fmax)

    return ws


def calculate_absxsec(
    ws, pressure=800e2, temperature=300.0,
):
    """Calculate absorption cross sections.

    Parameters:
        ws (Workspace): ARTS Workspace.
        pressure (float): Atmospheric pressure [Pa].
        temperature (float): Atmospheric temperature [K].

    Returns:
        ndarray, ndarray: Frequency grid [Hz], Abs. cross sections [m^2]
    """
    # Setting the pressure, temperature and vmr
    ws.NumericSet(ws.rtp_pressure, float(pressure))  # [Pa]
    ws.NumericSet(ws.rtp_temperature, float(temperature))  # [K]
    ws.VectorSet(ws.rtp_vmr, np.array([1.0]))  # [VMR]
    ws.Touch(ws.abs_nlte)

    ws.AbsInputFromRteScalars()

    # Calculate absorption cross sections
    ws.lbl_checkedCalc()
    ws.abs_xsec_agenda_checkedCalc()

    ws.abs_xsec_per_speciesInit()
    ws.abs_xsec_per_speciesAddLines()

    return ws.f_grid.value, ws.abs_xsec_per_species.value[0]

In [0]:
#plot_absorption()
ac = ArtsCalc()
interactive_plot = interactive(
    ac.plot_absorption,
    species=[basename(s).split(".")[0] for s in glob.iglob("./Perrin/*.xml.gz")],
    nfrequencies=(1000, 10000, 1),
    temperature=(200, 400, 1.),
    pressure=(0, 1000, 1.),
)
output = interactive_plot.children[-1]
interactive_plot